### Import Required Libraries and Set Up Environment Variables

In [ ]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import pandas as pd
from pandas import json_normalize

In [ ]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [ ]:


# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = ('print_headline', 'web_url', 'snippet', 'source', 'keyword', 'pub_date', 'byline', 'word_count')

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

#Build Url
base_query = (f"{url}api-key={nyt_api_key}&begin_date={begin_date}&end_date={end_date}"
    + f'&fq={filter_query}&sort={sort}&fl={field_list}')

In [ ]:

# List to store all reviews
all_reviews = []

# Loop through pages 0-19
for page in range(20):
    # Create query with a page number
    page_query = f"{base_query}&page={page}"

    # Make a "GET" request and retrieve the JSON
    response = requests.get(page_query)
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    if response.status_code == 200:
        # Parse the JSON response
        reviews = response.json()
        
        # Loop through the reviews["response"]["docs"] and append each review to the list
        if 'response' in reviews and 'docs' in reviews['response']:
            for review in reviews["response"]["docs"]:
                all_reviews.append(review)
            
            # Print the page that was just retrieved (moved outside the for loop)
            print(f"Retrieved page {page}")
    else:
        print(f"No 'docs' found in the response for page {page}")
        break


In [ ]:
reviews_list = response.json()

print((5, json.dumps(reviews_list, indent=4)))


In [ ]:


# Convert reviews_list to a Pandas DataFrame
df_reviews = json_normalize(reviews_list['response']['docs'])

In [ ]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
df_reviews['title'] = df_reviews['headline.main'].str.extract(r'\u2018(.*?)\u2019 Review')
df_reviews['title'] = df_reviews['title'] + ' Review'

print(df_reviews)

In [ ]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
df_reviews['keywords'] = df_reviews['keywords'].apply(lambda x: ', '.join([f"{item['name']}: {item['value']}" for item in x]))

print(df_reviews)

In [ ]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
tmdb_movies_list = df_reviews['title'].to_list()
print(tmdb_movies_list)

In [ ]:
#tmdb_movies_list = ['The Attachment Diaries',
 'What’s Love Got to Do With It?’ Probably a Lo',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterley’s Lover',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Love and Thunder',
 'Both Sides of the Blade',
 'Fire of Love',
 'Love & Gelato',
 'Stay Prayed Up',
 'Benediction',
 'Dinner in America',
 'In a New York Minute',
 'Anaïs in Love',
 'I Love America',
 'See You Then',
 'La Mami',
 'Love After Love',
 'Deep Water',
 'Lucy and Desi',
 'Cyrano',
 'The In Between',
 'Book of Love',
 'Lingui, the Sacred Bonds',
 'The Pink Cloud',
 'A Journal for Jordan',
 'West Side Story',
 'Aulcie',
 'Love Is Love Is Love',
 'Love Hard',
 'Bergman Island',
 'Hard Luck Love Song',
 'South of Heaven',
 'Wife of a Spy',
 'Happier Than Ever',
 'Together',
 'Annette',
 'Resort to Love',
 'Woodstock 99: Peace, Love and Rage',
 'Casanova, Last Love',
 'Running Against the Wind',
 'Asia',
 'Undine',
 'This Town',
 'Tu Me Manques',
 'Monday',
 'Ride or Die',
 'Future People',
 'Luz',
 'Happily',
 'This Is the Life',
 'To All the Boys: Always and Forever',
 'Young Hearts',
 'Little Fish',
 'Two of Us',
 'Atlantis',
 'Preparations to Be Together',
 'Your Name Engraved Herein',
 'Sylvie’s Love',
 'Ariana Grande: Excuse Me, I Love You',
 'Museum Town',
 'Wild Mountain Thyme',
 'My Psychedelic Love Story',
 '69: The Saga of Danny Hernandez',
 'Ammonite',
 'Love and Monsters',
 'Dick Johnson Is Dead',
 'Love, Guaranteed',
 'Feel the Beat',
 'Babyteeth',
 'Spelling the Dream',
 'A Secret Love',
 'Love Wedding Repeat',
 'Almost Love',
 'Hope Gap',
 'All the Bright Places',
 'The Photograph',
 'You Go to My Head',
 'Ordinary Love',
 'To All the Boys: P.S. I Still Love You',
 'The Woman Who Loves Giraffes',
 'Queen & Slim',
 'Marriage Story',
 'Cyrano, My Love',
 'Pretenders',
 'First Love',
 'Loro',
 'Falling Inn Love',
 'Hot Air',
 'Love, Antosha',
 'Leto',
 'The Tomorrow Man',
 'Asako I & II',
 'The Sun Is Also a Star',
 'Shéhérazade',
 'Long Shot',
 'Clara',
 'Kalank',
 'Rafiki',
 'Diane',
 'Five Feet Apart',
 'Ash Is Purest White',
 'Gloria Bell',
 'Black Mother',
 'The Hole in the Ground',
 'How to Train Your Dragon: The Hidden World',
 'A Tuba to Cuba',
 'Fighting With My Family',
 'Sorry Angel',
 'Berlin, I Love You',
 'Untogether',
 'Cold War',
 'If Beale Street Could Talk',
 'Asher',
 'The Party’s Just Beginning',
 'The Great Pretender',
 'Sicilian Ghost Story',
 'The New Romantic',
 'Pimp',
 'In a Relationship',
 'They’ll Love Me When I’m Dead’ Documents Orson Welles’s Last Fil',
 'Burning,’ Love Ignites a Divided Worl',
 'After Everything,’ a Young Love Blooms in Crisi',
 'Quincy’ Captures a Lifelong Love Affair With Musi',
 'Love, Gilda,’ a Portrait of a Brief and Brilliant Caree',
 'Tea With the Dames,’ Four Legends Dish on Acting and Lov',
 'Bel Canto,’ Music Is the Food of Love and Rebellio',
 'The Citizen,’ an Immigrant Picks a Bad Time to Fall in Lov',
 'Love, Cecil,’ an Aesthete Ahead of His Tim',
 'How to Talk to Girls at Parties',
 'Rogers Park,’ Life and Love in a Chicago Neighborhoo',
 'Love & Bananas,’ Uncovering the Plight of the Asian Elephan',
 'Godard Mon Amour',
 'Submergence,’ a Love Story Sunk by Geopolitic',
 'Love After Love,’ an Unflinching Look at Extended Grie',
 'Outside In',
 'Love, Simon,’ a Glossy Teen Romance, the Hero Has a Secre',
 'Keep the Change,’ Two People With Autism Find Lov',
 'The Party',
 'The Housemaid,’ Twisted Love and Angry Ghosts in Vietna',
 'Forever My Girl,’ a Romance Resumes After a Long Brea',
 'Kangaroo: A Love-Hate Story’ Exposes a Wildlife Massacr',
 'Lover for a Day',
 'Youth,’ the People’s Dance Troupe, in Love and Wa',
 'Killing for Love’ Revisits a Virginia Murder Cas',
 'The Shape of Water’ Is Altogether Wonderfu',
 'Love Beats Rhymes,’ a Hip-Hop Artist Transformed by Poetr',
 'Cuba and the Cameraman’ Lavishes Love on a Country … and Castr',
 'On the Beach at Night Alone’ Zooms in on a Love Affai',
 'Thelma,’ a Woman in Love Can Burn Down the Worl',
 'Hello Again,’ a Movie Musical Ode to Love and Lust Over Decade',
 'It Happened in L.A.,’ All That Questing After Lov',
 'God’s Own Country',
 'The Mountain Between Us',
 'Dina,’ a Differently Abled Love Stor',
 'In Search of Fellini',
 'Woodpeckers,’ a Tale of Love and Agonizing Penal Confinemen',
 'I Do ... Until I Don’t,’ Love and Loathing in Florid',
 'Tales of an Immoral Couple’: Love Means Having to Grow U',
 'After Love’ and Regretting Every Momen',
 'The Last Face',
 'Women Who Kill,’ and May Be in Love With On',
 'False Confessions,’ the Play’s Not Quite the Thin',
 'Review: Those Movies, Himself — Bertrand Tavernier’s Tour of French Cinem',
 'The Big Sick,’ Comedy Is Hard, Love Harde',
 'Lost in Paris',
 'Vincent N Roxxy,’ Love Is in the Air, N So Is Fea',
 'Everything, Everything’ Pits Love Against Diseas',
 'Hounds of Love’ Is Tense and Deadly Down Unde',
 'Harold and Lillian’ Introduces a Hollywood Power Coupl',
 'The Happiest Day in the Life of Olli Maki,’ Love and Life on the Rope',
 'The Promise’ Finds a Love Triangle in Constantinopl',
 'Frantz,’ a Mysterious Frenchman and the Wounds of Wa',
 'The Other Half',
 'The Ottoman Lieutenant',
 'Love & Taxes',
 'Everybody Loves Somebody,’ a Rom-Com With Bit',
 'Kedi,’ Rekindling a ‘Love of Life']

### Access The Movie Database API

In [ ]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = f"&api_key={tmdb_api_key}"


In [ ]:
new_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 0

# Loop through the titles
for title in tmdb_movies_list:
    # Check if we need to sleep before making a request
    if request_counter % 50 == 0 and request_counter != 0:
        print(f"Sleeping for 10 seconds...")
        time.sleep(10)
    
    # Add 1 to the request counter
    request_counter += 1
    
    # Perform a "GET" request for The Movie Database
    response = requests.get(url + title + tmdb_key_string).json()
    
    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
        # Get movie id
        movie_id = response['results'][0]['id']
        
        # Make a request for the full movie details
        movie_details = requests.get(f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}").json()

        
        # Extract the genre names into a list
        genres = [genre['name'] for genre in movie_details['genres']]
        
        # Extract the spoken_languages' English name into a list
        spoken_languages = [language['english_name'] for language in movie_details['spoken_languages']]
        
        # Extract the production_countries' name into a list
        production_countries = [country['name'] for country in movie_details['production_countries']]
        
        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        movie_data = {
            'title': title,
            'genres': genres,
            'spoken_languages': spoken_languages,
            'production_countries': production_countries
        }
        new_list.append(movie_data)
        
        # Print out the title that was found
        print(f"Found movie: {title}")
    except:
        print(f"Movie not found: {title}")



In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print((5, json.dumps(tmdb_movies_list, indent=4)))


In [ ]:
# Convert the results to a DataFrame
df_tmdb_movies = pd.DataFrame(tmdb_movies_list)
df_tmdb_movies.head()

### Merge and Clean the Data for Export

In [ ]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(df_reviews, df_tmdb_movies, how="inner", on="title")

In [ ]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ['column_1', 'column_2', 'column_3']

# Create a list of characters to remove
characters_to_remove = ['[', ']', '"']

# Loop through the list of columns to fix
for column in columns_to_fix:
    # Convert the column to type 'str'
    df[column] = df[column].astype(str)

    # Loop through characters to remove
    for chr in characters_to_remove:
        df[column] = df[column].str.replace(chr, '')
# Display the fixed DataFrame
df(merged_df)

In [ ]:

# Drop "byline.person" column
df.drop("byline.person", axis=1, inplace=True)

In [ ]:
# Delete duplicate rows
df.drop_duplicates(inplace=True)

# Reset index
df.reset_index(drop=True, inplace=True)


In [ ]:
df.to_csv('filename.csv', index=False)
